In [1]:
import pandas as pd

pd.options.plotting.backend = "plotly"
import numpy as np
import plotly.graph_objects as go


from ares_py.class_project import Project
from ares_py.tools.get_ld import get_ld

**CREATE PROJECT**

1. Run init project with project name input -> prj class, creates folder structure
1. Run prj.Process_2dm() - with **flat mode coordinates**


In [2]:
prj = Project("kysuce", crs=8353, res_range=[1, 1000])
prj = prj.Process_2dm(crd_mode="flat")

projects\kysuce\01_input\001.2dm
projects\kysuce\01_input\002.2dm
projects\kysuce\01_input\003.2dm
projects\kysuce\01_input\004.2dm
projects\kysuce\01_input\005.2dm
projects\kysuce\01_input\006.2dm


**PROCESS - ert_plan_pt**

1. Draw planed lines - **ert.gpkg - ert_plan_ls**
1. Copy **dtm.tif** into project/qgis folder
1. Run model **ert_ls2pt** in QGIS project and save data manually to **./tmp/tmp_plan.gpkg**
1. Run prj.Process_crd_plan () -> **input/{line}_plan.csv**, **ert.gpkg - ert_plan_pt**
1. Run prj.Process_2dm() -> loads 2dm, adds coordinates from **input/{line}_plan.csv**


In [3]:
prj = prj.Process_crd_plan()
prj = prj.Process_2dm(crd_mode="plan")

projects\kysuce\01_input\001.2dm
projects\kysuce\01_input\002.2dm
projects\kysuce\01_input\003.2dm
projects\kysuce\01_input\004.2dm
projects\kysuce\01_input\005.2dm
projects\kysuce\01_input\006.2dm


**PROCESS man coordinates**
1. Run prj.Process_crd_rec() -> **crd_man_input_pt** *(if not exist)*
1. Edit man pt coordinates **ert.gpkg - crd_man_input_pt**
1. Run model rec proc_man  and save to  **tmp/tmp_rec.gpkg**  *(interpolated (1m) coordinates from ert.gpkg - crd_man_input_pt, DTM sampled)*
1. Run prj.Process_crd_man() -> **crd_man_proc_pt** *(coordinates with recalculated ld)*, **input/{line}_man.csv**

In [4]:
prj = prj.Process_crd_rec()
prj = prj.Process_crd_man()

c:\Users\adamg\AppData\Local\Programs\Python\Python313\Lib\site-packages\pyogrio\raw.py:198: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D Point' is converted to 'Point Z'
  return ogr_read(
c:\Users\adamg\AppData\Local\Programs\Python\Python313\Lib\site-packages\pyogrio\raw.py:198: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D Point' is converted to 'Point Z'
  return ogr_read(


Processing rec crd - line 1, 12 entries
Processing rec crd - line 2, 17 entries
Processing rec crd - line 3, 12 entries
Processing rec crd - line 4, 5 entries
Processing rec crd - line 5, 12 entries
Processing rec crd - line 6, 6 entries


**PROCESS 2dm**

1. Set coordinates mode to select coordinates type from csv file:
    + *plan - original planned coordinates*
    + *rec - recorded in field*
    + *man - manual coordinates*
1. Run prj.Process_2dm() -> loads 2dm, adds coordinates from **input/{line}_man.csv**
1. Run prj.Proc_topo_2d() -> exports topo to **ert.gpkg -ert2d_topo_pt, ert2d_topo_ls**


In [5]:
prj = prj.Process_2dm(crd_mode="man")
prj = prj.Proc_topo_2d()

projects\kysuce\01_input\001.2dm
projects\kysuce\01_input\002.2dm
projects\kysuce\01_input\003.2dm
projects\kysuce\01_input\004.2dm
projects\kysuce\01_input\005.2dm
projects\kysuce\01_input\006.2dm


**Process inversion outputs**
1. Do the inversions - export data to **inv/** folder formats:
    + zond - _znd.dat
    + r2d - _topres.dat

1. Run prj.Export_gridding_input() - reads and formats inversion outputs -> **04_grd/{line}.tsv**
2. Run prj.Export_gpkg_inv() - exports inversion points to -> **ert.gpkg - inv_{type}_pt**
    + type = r2d / znd
1. Run prj.Grid_mask()- exports measured da ->  **ert.gpkg - ert2d_grd_mask_pl**


In [6]:
prj = prj.Export_gridding_input()
prj = prj.Export_gpkg_inv()
prj = prj.Grid_mask()

Exported...projects\kysuce\04_grd\001_r2d.tsv
Exported...projects\kysuce\04_grd\001_znd.tsv
Exported...projects\kysuce\04_grd\002_r2d.tsv
Exported...projects\kysuce\04_grd\002_znd.tsv
Exported...projects\kysuce\04_grd\003_r2d.tsv
Exported...projects\kysuce\04_grd\003_znd.tsv
Exported...projects\kysuce\04_grd\004_r2d.tsv
Exported...projects\kysuce\04_grd\004_znd.tsv
Exported...projects\kysuce\04_grd\005_r2d.tsv
Exported...projects\kysuce\04_grd\005_znd.tsv
Exported...projects\kysuce\04_grd\006_r2d.tsv
Exported...projects\kysuce\04_grd\006_znd.tsv


**Grid data**
1. Grid grd inputs manually or run prj.Grid_data() to grid and export surfer grids 
    * enter skip list to ignore lines (eg. for long lines)
    * runs gridding on all .tsv files in **04_grd/** (except skipped lines)
    * -> **04_grd/{line}_{input_name}.grd


In [7]:
# prj = prj.Grid_data(skip=[], cell_size=1)

**Process contours**
1. Run batch process countours from desired grids into separate file
    * naming -**ares_py/tmp/tmp_cnt{line}.gpkg**
2. Run prj.Merge_contours() below to merge separate files into single file (**cnt_{type}_full_ls**) for selected **type** of inversion. 
3. Run mask to **grd_mask_pl** or **grd_mask_man_pl** after
4. Delete individual files


In [8]:
prj = prj.Merge_contours(type="r2d")

Empty DataFrame
Columns: [fp, fn, f, ext, fp_abs]
Index: []


ValueError: No objects to concatenate

**Generate atlas_qc:**
1. Enter atlas input: 
    + *res_range_qc - res_min and res_max for qc graphs in Ohmm*
    + *page heights*
    + *map scales*
    + *lvl_frequency = layer frequency for qc graphs 1= all, 2= every 2nd...*
    + *z_gap = offset from top wof atlas extent for grid maps in meters*
1. Run prj.Atlas_qc() -> **ert.gpkg - atlas_qc**
1. Run prj.QC_lines() -> **qgis/{line}.png, qgis/{line}.pgw,** 

In [9]:
# input - dd_lavka
# atlas_input = dict(
#     res_min=[100] * 5,
#     res_max=[20000] * 5,
#     page_heights=[594, 420, 420, 420, 420],
#     map_scales=[1000] * 5,
#     lvl_frequency=5 * [3],
#     z_gap=[15, 15, 15, 15, 5],
# )

atlas_input = dict(
    res_min=[0] * 6,
    res_max=[1000] * 6,
    page_heights=[420] * 6,
    map_scales=[500] * 6,
    lvl_frequency=6 * [3],
    z_gap=[15] * 6,
)


prj = prj.Atlas_qc(atlas_input)
prj = prj.QC_lines()
pd.DataFrame(prj.atlas_qc)


prj = prj.Atlas_qc(atlas_input)
prj = prj.QC_lines()
pd.DataFrame(prj.atlas_qc)

projects\kysuce\qgis\001.png
projects\kysuce\qgis\002.png
projects\kysuce\qgis\003.png
projects\kysuce\qgis\004.png
projects\kysuce\qgis\005.png
projects\kysuce\qgis\006.png
projects\kysuce\qgis\001.png
projects\kysuce\qgis\002.png
projects\kysuce\qgis\003.png
projects\kysuce\qgis\004.png
projects\kysuce\qgis\005.png
projects\kysuce\qgis\006.png


,res_min,res_max,page_heights,map_scales,lvl_frequency,z_gap,xrange,zmax,el_space,xmin,xmax,map_height,zmin,zrange,page_width,geometry
ID_line,,,,,,,,,,,,,,,,
1,0,1000,420,500,3,15,471.87,395.0,3,10000,10471.87,130.0,330.0,65.0,978.74,"POLYGON ((10000 330, 10471.87 330, 10471.87 39..."
2,0,1000,420,500,3,15,468.64,415.0,3,20000,20468.64,130.0,350.0,65.0,972.28,"POLYGON ((20000 350, 20468.64 350, 20468.64 41..."
3,0,1000,420,500,3,15,256.82,410.0,3,30000,30256.82,130.0,345.0,65.0,548.64,"POLYGON ((30000 345, 30256.82 345, 30256.82 41..."
4,0,1000,420,500,3,15,155.30,415.0,2,40000,40155.30,130.0,350.0,65.0,345.60,"POLYGON ((40000 350, 40155.3 350, 40155.3 415,..."
5,0,1000,420,500,3,15,107.33,425.0,2,50000,50107.33,130.0,360.0,65.0,249.66,"POLYGON ((50000 360, 50107.33 360, 50107.33 42..."
6,0,1000,420,500,3,15,141.91,415.0,2,60000,60141.91,130.0,350.0,65.0,318.82,"POLYGON ((60000 350, 60141.91 350, 60141.91 41..."


In [ ]:
from ares_py.tools.get_ld import get_ld
from ares_py.tools.colors import get_ref_res

import geopandas as gpd


ld = get_ld(prj.fps["grd"], ext=".tsv")
ld["type"] = ld["f"].str.slice(4, None)
ld["ID_line"] = ld["f"].str.slice(None, 3).astype(int)


for type in ld["type"].unique():
    ld_type = ld.copy()
    ld_type = ld_type.loc[ld_type["type"] == type]

    gdf = []
    for fp, line in zip(ld_type["fp"], ld_type["ID_line"]):
        df = pd.read_csv(fp, sep="\t")
        df["ID_line"] = line
        gdf.append(df)

    gdf = pd.concat(gdf).reset_index(drop=True)
    gdf.columns = ["ld_hor", "z", "res", "res_log", "ID_line"]
    gdf["ld_hor"] += gdf["ID_line"] * 10000
    geom = gpd.points_from_xy(gdf["ld_hor"], gdf["z"])
    gdf = gpd.GeoDataFrame(gdf, geometry=geom, crs=prj.crs)

    gdf["ref_res"] = get_ref_res(gdf["res"], prj.res_range)
    gdf = prj.Merge_el_space(gdf)
    gdf["grd1"] = gdf.loc[gdf["ID_line"] == 1, "res"]

    gdf.to_file(prj.fps["ert"], layer=f"inv_{type}_pt", engine="pyogrio")